In [ ]:
import os
import reservoirpy as rpy
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
from sklearn.metrics import r2_score
from reservoirpy.nodes import Reservoir, Ridge
import logging
import gc
import pickle

# ログ設定
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# グローバル設定
rpy.verbosity(0)  # Verboseモードを無効
rpy.set_seed(42)  # 再現性のためのシード設定

In [ ]:
# パラメータ設定
n_timesteps = 3200
units = 2000
rc_connectivity_list = np.array([0.2, 0.4, 0.6, 0.8, 1.0])
spectral_radii = np.array([0.1, 0.25, 0.5, 0.75, 1.0, 1.25])
max_delay = 3200  # 遅延数
batch_size = 20  # バッチサイズを最適化

# 結果保存先のディレクトリ
output_dir = "jupyter/src/memory_capacity/results/modulerESN_MC"
os.makedirs(output_dir, exist_ok=True)  # ディレクトリがなければ作成

In [ ]:
# データ生成
np.random.seed(42)
input_signal = np.random.randn(n_timesteps).reshape(-1, 1)  # ランダムな入力信号を生成
input_signal = (input_signal - np.mean(input_signal)) / np.std(input_signal)  # 標準化
X = [input_signal[:-d] for d in range(1, max_delay + 1)]
Y = [input_signal[d:] for d in range(1, max_delay + 1)]

In [ ]:
def create_4modular_esn(spectral_radius, rc_connectivity, n_modules=4):
    modules = [Reservoir(units=units//n_modules, sr=spectral_radius, rc_connectivity=rc_connectivity) for _ in range(n_modules)]
    readout = Ridge(output_dim=1, ridge=1e-5)
    return [modules[0],modules[1],modules[2],modules[3]] >> readout

def create_4modular_esn_inter(spectral_radius, rc_connectivity, n_modules=4):
    inter_connectivity = (rc_connectivity + 1) / 2
    modules = [Reservoir(units=units//n_modules, sr=spectral_radius, rc_connectivity=rc_connectivity, inter_connectivity=inter_connectivity) for _ in range(n_modules)]
    readout = Ridge(output_dim=1, ridge=1e-5)
    return [modules[0],modules[1],modules[2],modules[3]] >> readout

In [ ]:
# R^2スコアを計算
def compute_r2(spectral_radius, rc_connectivity, is_inter, input_signal, output_signal):
    try:
        if is_inter:
            esn = create_4modular_esn_inter(spectral_radius, rc_connectivity)
        else:
            esn = create_4modular_esn(spectral_radius, rc_connectivity)
        esn.fit(input_signal, output_signal)
        pred = esn.run(input_signal)
        r2 = r2_score(output_signal, pred)
        del esn, pred
        gc.collect()
        return max(r2, 0)
    except Exception as e:
        logging.error(f"Failed to compute R2 score for sr={spectral_radius}, rc_connectivity={rc_connectivity}, is_inter={is_inter}: {e}")
        return np.nan

# メモリ容量を計算
def compute_memory_capacity(spectral_radius, rc_connectivity, is_inter, X, Y):
    logging.info(f"Calculating memory capacity for spectral radius={spectral_radius}, rc_connectivity={rc_connectivity} and is_inter={is_inter}")
    r2_scores = []
    try:
        for start in range(0, max_delay, batch_size):
            end = min(start + batch_size, max_delay)
            r2_batch = Parallel(n_jobs=8, backend="loky")(delayed(compute_r2)(
                spectral_radius, rc_connectivity, is_inter, X[d], Y[d]) for d in range(start, end))
            r2_scores.extend(r2_batch)
            gc.collect()  # 各バッチ後にメモリ解放
        memory_capacity = np.nansum(r2_scores)
        return {
            "Is Inter": is_inter,
            "Spectral Radius": spectral_radius,
            "RC Connectivity": rc_connectivity,
            "Memory Capacity": memory_capacity,
            "R2 Scores": r2_scores
        }
    except Exception as e:
        logging.error(f"Failed to compute memory capacity for spectral radius={spectral_radius}, rc_connectivity={rc_connectivity} and is_inter={is_inter}: {e}")
        return None

# 部分保存関数
def save_partial_results(result):
    try:
        spectral_radius = result["Spectral Radius"]
        rc_connectivity = result["RC Connectivity"]
        is_inter = str(result["Is Inter"])
        file_name = f"sr_{spectral_radius}_rc_{rc_connectivity}_is_inter_{is_inter}.pkl"
        path_name = os.path.join(output_dir, file_name)
        with open(path_name, "wb") as f:
            pickle.dump(result, f)
        logging.info("Partial result saved successfully.")
    except Exception as e:
        logging.error(f"Failed to save partial result: {e}")

# 全てのパラメータの組み合わせについてメモリ容量を計算
def compute_all_memory_capacities(spectral_radii, rc_connectivity_list, X, Y):
    for sr in spectral_radii:
        for rc_connectivity in rc_connectivity_list:
            for is_inter in [True, False]:
                result = compute_memory_capacity(sr, rc_connectivity, is_inter, X, Y)
                if result is not None:
                    save_partial_results(result)


In [ ]:
# 実行
compute_all_memory_capacities(spectral_radii, rc_connectivity_list, X, Y)
print("All memory capacity results have been calculated and saved.")